# Stacking:
1. we combine out classifiers to one and also try hyperparameter tuning on the base and meta models.

**desired output**:

test prediction to upload

## Import Libraries

In [49]:
!pip install category_encoders

In [50]:
# libraries
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.ensemble import  AdaBoostClassifier,  ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier 
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


from category_encoders.basen import BaseNEncoder
from category_encoders.count import CountEncoder

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statistics
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from sklearn.preprocessing import MinMaxScaler

In [51]:
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.woe import WOEEncoder
from category_encoders.target_encoder import TargetEncoder
from category_encoders.sum_coding import SumEncoder
from category_encoders.m_estimate import MEstimateEncoder
from category_encoders.leave_one_out import LeaveOneOutEncoder
from category_encoders.helmert import HelmertEncoder
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.james_stein import JamesSteinEncoder
from category_encoders.one_hot import OneHotEncoder
from category_encoders.basen import BaseNEncoder
from category_encoders.count import CountEncoder
from category_encoders.glmm import GLMMEncoder
from sklearn.preprocessing import MinMaxScaler

In [52]:
data_all_filtered= pd.read_csv('data/data_filtered.csv')
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
data_all_filtered.drop(columns=['Cabin_location'],inplace=True)
data_all_filtered.shape, train.shape, test.shape 

((1309, 13), (891, 12), (418, 11))

In [53]:
data_all_filtered.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,is_Alone,Embarked,Fare_bin,Family_size,Title,age_bin
0,0.0,3,0,22.0,1,0,7.2500,0,S,cheap,medium,Mr,adult
1,1.0,1,1,38.0,1,0,71.2833,0,C,very_high,medium,Mrs,adult
2,1.0,3,1,26.0,0,0,7.9250,1,S,cheap,single,Miss,adult


In [54]:
def split_trainANDtest(data_all_filtered):
  data_train = data_all_filtered.iloc[0:train.shape[0],:]
  data_test = data_all_filtered.iloc[train.shape[0]:,:]
  return data_train, data_test


def Min_Max_Scaler_X(train_enc, val_enc):
  scaler = MinMaxScaler()
  train_enc = scaler.fit_transform(train_enc)
  val_enc = scaler.transform(val_enc)
  return train_enc, val_enc

def train_clasifier(classifier_X,X_train,y_train,X_val,y_val):
  baseline = classifier_X
  scores_cv = cross_val_score(baseline, X_train, y_train, cv=5,scoring='accuracy')
  model = baseline.fit(X_train,y_train)
  scores_pred = accuracy_score(y_val,model.predict(X_val))
  return scores_cv, scores_pred


classifiers_list = [
      XGBClassifier(random_state=42), 
      RandomForestClassifier(random_state=42), 
      LGBMClassifier(random_state=42),   
      KNeighborsClassifier(), 
      SVC()
                          ]

#import from preous notebook
classifer0_cols = ['Pclass', 'Sex', 'Age', 'Parch', 'Fare', 'is_Alone', 'Fare_bin', 'Family_size', 'Title']
classifer1_cols =['Pclass', 'Sex', 'Age', 'SibSp', 'is_Alone', 'Fare_bin', 'Family_size', 'Title', 'age_bin']
classifer2_cols =['Pclass', 'Sex', 'Age', 'Fare', 'is_Alone', 'Embarked', 'Fare_bin', 'Family_size', 'Title']
classifer3_cols =['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'is_Alone', 'Embarked', 'Fare_bin', 'Family_size', 'Title']
classifer4_cols =['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_bin', 'Title', 'age_bin']

#import from preous notebook
label_encoder_cols =['Fare_bin', 'Embarked']
counter_encoder_cols = ['Family_size', 'Title', 'age_bin']
print(label_encoder_cols, counter_encoder_cols)

['Fare_bin', 'Embarked'] ['Family_size', 'Title', 'age_bin']


In [55]:
from category_encoders.one_hot import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from mlxtend.classifier import StackingClassifier
from sklearn import model_selection
from sklearn.pipeline import make_pipeline
from mlxtend.feature_selection import ColumnSelector
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from mlxtend.feature_selection import ColumnSelector

import warnings

warnings.simplefilter('ignore')

## 1. Train/test baseline stack

In [56]:
data_train, data_test = split_trainANDtest(data_all_filtered)
data_train.shape, data_test.shape
#test
X_train, X_val, y_train, y_val = train_test_split(data_train.drop(columns=['Survived']),data_train['Survived'],random_state=42,test_size=0.2, shuffle=True)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

X_train['Fare_bin'] = X_train['Fare_bin'].map({'cheap':0,'medium':1,'high':2,'very_high':3})
X_train['Embarked'] = X_train['Embarked'].map({'S':0,'C':1,'Q':2})

X_val['Fare_bin'] = X_val['Fare_bin'].map({'cheap':0,'medium':1,'high':2,'very_high':3})
X_val['Embarked'] = X_val['Embarked'].map({'S':0,'C':1,'Q':2})

best_encoder = CountEncoder(cols=counter_encoder_cols,return_df=True, handle_unknown=0)
X_train = best_encoder.fit_transform(X_train,y_train)
X_val = best_encoder.transform(X_val)

(712, 12) (712,) (179, 12) (179,)


In [57]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,is_Alone,Embarked,Fare_bin,Family_size,Title,age_bin
331,1,0,46.0,0,0,28.5,1,0,1,429,419,502
733,2,0,23.0,0,0,13.0,1,0,1,429,419,502


In [58]:
X_val.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,is_Alone,Embarked,Fare_bin,Family_size,Title,age_bin
709,3,0,17.0,1,1,15.2458,0,1,1,206,33,104
439,2,0,31.0,0,0,10.5000,1,0,1,429,419,502


In [59]:
clf1 = XGBClassifier(random_state=42)
clf2 = RandomForestClassifier(random_state=42)
clf3 = LGBMClassifier(random_state=42)
clf4 = KNeighborsClassifier() 
clf5 = SVC(probability=True,random_state=42)
lr = LogisticRegression(random_state=42)

pipe1 = make_pipeline(ColumnSelector(cols=classifer0_cols),  MinMaxScaler(), clf1 )
pipe2 = make_pipeline( ColumnSelector(cols=classifer1_cols), MinMaxScaler(), clf2 )
pipe3 = make_pipeline( ColumnSelector(cols=classifer2_cols), MinMaxScaler(), clf3 )
pipe4 = make_pipeline(ColumnSelector(cols=classifer3_cols), MinMaxScaler() , clf4 )
pipe5 = make_pipeline(ColumnSelector(cols=classifer4_cols), MinMaxScaler() , clf5 )
lr = LogisticRegression()

sclf_baseline = StackingClassifier(classifiers=[pipe1, pipe2, pipe3,pipe4,pipe5], 
                          meta_classifier=lr,use_probas=True, average_probas=False)
print('5-fold cross validation:\n')

for clf, label in zip([pipe1, pipe2, pipe3,pipe4,pipe5,sclf_baseline], 
                      ['XGBoost', 
                       'Random Forest', 
                       'LGBMClassifier',
                       'KNeighbors',
                       'SVC',
                       'StackingClassifier'
                       ]):

    scores = model_selection.cross_val_score(clf, X_train, y_train, 
                                              cv=5, scoring='accuracy')
    print("Accuracy: %0.3f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))

5-fold cross validation:

Accuracy: 0.833 (+/- 0.04) [XGBoost]
Accuracy: 0.806 (+/- 0.02) [Random Forest]
Accuracy: 0.809 (+/- 0.02) [LGBMClassifier]
Accuracy: 0.816 (+/- 0.02) [KNeighbors]
Accuracy: 0.824 (+/- 0.02) [SVC]
Accuracy: 0.801 (+/- 0.02) [StackingClassifier]


In [60]:
pipe1.fit(X_train,y_train)
pipe2.fit(X_train,y_train)
pipe3.fit(X_train,y_train)
pipe4.fit(X_train,y_train)
pipe5.fit(X_train,y_train)
sclf_baseline.fit(X_train,y_train)

pipe1_pred = pd.DataFrame(pipe1.predict(X_val),columns=['XGBoost'])
pipe2_pred = pd.DataFrame( pipe2.predict(X_val),columns=['RF'])
pipe3_pred = pd.DataFrame(pipe3.predict(X_val),columns=['LGBM'])
pipe4_pred = pd.DataFrame(pipe4.predict(X_val),columns=['KNN'])
pipe5_pred = pd.DataFrame(pipe5.predict(X_val),columns=['SVC'])
sclf_baseline_pred = pd.DataFrame(sclf_baseline.predict(X_val), columns=['Stacked'])
baseline_df = pd.concat([pipe1_pred,pipe2_pred,pipe3_pred,pipe4_pred,pipe5_pred,sclf_baseline_pred],axis=1).astype(int)

In [61]:
baseline_df.head()

,XGBoost,RF,LGBM,KNN,SVC,Stacked
0,1,1,1,1,0,1
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,1,1,1,1,1,1
4,0,0,1,0,1,0


In [62]:
result_df = pd.DataFrame({'classifiers': ['XGBoost','RF','LGBM','KNN','SVC','Stacked'],
             'accuracy': [
                          accuracy_score(y_val, pipe1.predict(X_val)),
                          accuracy_score(y_val, pipe2.predict(X_val)) ,
                          accuracy_score(y_val, pipe3.predict(X_val)) ,
                          accuracy_score(y_val, pipe4.predict(X_val)) ,
                          accuracy_score(y_val, pipe5.predict(X_val)) ,
                          accuracy_score(y_val, sclf_baseline.predict(X_val)) 
                          ]}
             )
result_df

,classifiers,accuracy
0,XGBoost,0.854749
1,RF,0.860335
2,LGBM,0.871508
3,KNN,0.821229
4,SVC,0.810056
5,Stacked,0.849162


## 2. Full: baseline stack

In [63]:
data_train, data_test = split_trainANDtest(data_all_filtered)
data_train.shape, data_test.shape

X_train_full, y_train_full, X_test_full = data_train.drop(columns=['Survived']),    data_train['Survived'], data_test.drop(columns=['Survived'])
X_train_full.shape, y_train_full.shape, X_test_full.shape

X_train_full['Fare_bin'] = X_train_full['Fare_bin'].map({'cheap':0,'medium':1,'high':2,'very_high':3})
X_train_full['Embarked'] = X_train_full['Embarked'].map({'S':0,'C':1,'Q':2})

X_test_full['Fare_bin'] = X_test_full['Fare_bin'].map({'cheap':0,'medium':1,'high':2,'very_high':3})
X_test_full['Embarked'] = X_test_full['Embarked'].map({'S':0,'C':1,'Q':2})

best_encoder = CountEncoder(cols=counter_encoder_cols,return_df=True, handle_unknown=0)
X_train_full = best_encoder.fit_transform(X_train_full,y_train_full)
X_test_full = best_encoder.transform(X_test_full)

In [64]:
X_train_full.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,is_Alone,Embarked,Fare_bin,Family_size,Title,age_bin
0,3,0,22.0,1,0,7.2500,0,0,0,263,517,626
1,1,1,38.0,1,0,71.2833,0,1,3,263,125,626


In [65]:
X_test_full.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,is_Alone,Embarked,Fare_bin,Family_size,Title,age_bin
891,3,0,34.0,0,0,7.8292,1,2,0,537,517,626
892,3,1,47.0,1,0,7.0000,0,0,0,263,125,626


In [66]:
clf1 = XGBClassifier(random_state=42)
clf2 = RandomForestClassifier(random_state=42)
clf3 = LGBMClassifier(random_state=42)
clf4 = KNeighborsClassifier() 
clf5 = SVC(probability=True,random_state=42)
lr = LogisticRegression(random_state=42)

pipe1_full = make_pipeline(ColumnSelector(cols=classifer0_cols),  MinMaxScaler(), clf1 )
pipe2_full = make_pipeline( ColumnSelector(cols=classifer1_cols), MinMaxScaler(), clf2 )
pipe3_full = make_pipeline( ColumnSelector(cols=classifer2_cols), MinMaxScaler(), clf3 )
pipe4_full = make_pipeline(ColumnSelector(cols=classifer3_cols), MinMaxScaler() , clf4 )
pipe5_full = make_pipeline(ColumnSelector(cols=classifer4_cols), MinMaxScaler() , clf5 )
lr = LogisticRegression()

sclf_baseline_full = StackingClassifier(classifiers=[pipe1, pipe5], 
                          meta_classifier=lr,use_probas=True, average_probas=False)

pipe1_full.fit(X_train,y_train)
pipe2_full.fit(X_train,y_train)
pipe3_full.fit(X_train,y_train)
pipe4_full.fit(X_train,y_train)
pipe5_full.fit(X_train,y_train)
sclf_baseline_full.fit(X_train,y_train)

pipe1_pred_full= pd.DataFrame(pipe1_full.predict(X_test_full),columns=['XGBoost'])
pipe2_pred_full = pd.DataFrame(pipe2_full.predict(X_test_full),columns=['RF'])
pipe3_pred_full = pd.DataFrame(pipe3_full.predict(X_test_full),columns=['LGBM'])
pipe4_pred_full = pd.DataFrame(pipe4_full.predict(X_test_full),columns=['KNN'])
pipe5_pred_full = pd.DataFrame(pipe5_full.predict(X_test_full),columns=['SVC'])
sclf_baseline_pred_full = pd.DataFrame(sclf_baseline_full.predict(X_test_full), columns=['Stacked'])
baseline_df_full = pd.concat([pipe1_pred_full,pipe2_pred_full,pipe3_pred_full,pipe4_pred_full,pipe5_pred_full,sclf_baseline_pred_full],axis=1).astype(int)

In [67]:
baseline_df_full.head()

,XGBoost,RF,LGBM,KNN,SVC,Stacked
0,0,0,0,0,0,0
1,0,0,0,0,1,0
2,0,1,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,1,0


In [68]:
sabmision = pd.read_csv('data/gender_submission.csv')
sabmision['Survived'] = baseline_df_full['Stacked'].values
sabmision['Survived'] = sabmision['Survived'].astype(int)
sabmision.to_csv('submision.csv',index=False)

# --------------------------------------------------------------------------------------------------------------------------------

## Stack with sklearn Stacker

In [69]:
clf1 = XGBClassifier()
clf2 = RandomForestClassifier()
clf3 = LGBMClassifier()
clf4 = KNeighborsClassifier() 
clf5 = SVC(probability=True)
lr = LogisticRegression()
pipe1_full = make_pipeline(ColumnSelector(cols=classifer0_cols),  MinMaxScaler(), clf1 )
pipe2_full = make_pipeline( ColumnSelector(cols=classifer1_cols), MinMaxScaler(), clf2 )
pipe3_full = make_pipeline( ColumnSelector(cols=classifer2_cols), MinMaxScaler(), clf3 )
pipe4_full = make_pipeline(ColumnSelector(cols=classifer3_cols), MinMaxScaler() , clf4 )
pipe5_full = make_pipeline(ColumnSelector(cols=classifer4_cols), MinMaxScaler() , clf5 )

In [70]:
from sklearn.ensemble import StackingClassifier
estimators = [
 ('xgb', pipe1_full ),
  ('RF', pipe2_full),
  ('lgbm', pipe3_full),
  ('knn', pipe4_full),
  ('svc', pipe5_full),
]
clf = StackingClassifier(
estimators=estimators, final_estimator=LogisticRegression()
)
pipe1_full.fit(X_train,y_train)
pipe2_full.fit(X_train,y_train)
pipe3_full.fit(X_train,y_train)
clf.fit(X_train,y_train)

pipe1_pred_full= pd.DataFrame(pipe1_full.predict(X_test_full),columns=['XGBoost'])
pipe2_pred_full = pd.DataFrame(pipe2_full.predict(X_test_full),columns=['LGBM'])
pipe3_pred_full = pd.DataFrame(pipe3_full.predict(X_test_full),columns=['SVC'])
clf_pred_full = pd.DataFrame(clf.predict(X_test_full), columns=['Stacked'])
baseline_df_full = pd.concat([pipe1_pred_full,pipe2_pred_full,pipe3_pred_full,clf_pred_full],axis=1).astype(int)

In [71]:
baseline_df_full.head()

,XGBoost,LGBM,SVC,Stacked
0,0,0,0,0
1,0,0,0,0
2,0,1,0,0
3,0,0,0,0
4,0,0,0,0


In [72]:
sabmision = pd.read_csv('data/gender_submission.csv')
sabmision['Survived'] = baseline_df_full['Stacked'].values
sabmision['Survived'] = sabmision['Survived'].astype(int)
sabmision.to_csv('submision_9_sklearn_stack.csv',index=False)

## Future Work:
- it can be tried GridSearchCV.
- different combination of classifirs can be tried.
